In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import re
# from database import load_films_to_db

# Base URL for Wikipedia
BASE_URL = "https://en.wikipedia.org"
HEADERS = {'User-Agent': 'Mozilla/5.0'}

def get_films_soup():
    """
    Fetch the Wikipedia page containing the list of highest-grossing films
    and return its parsed BeautifulSoup object.
    """
    response = requests.get(BASE_URL + '/wiki/List_of_highest-grossing_films', headers=HEADERS)
    return BeautifulSoup(response.content, "lxml")

soup = get_films_soup()

def extract(title_link):
    """
    Extracts key information from a given film's Wikipedia page.
    
    Parameters:
        title_link (str): The URL of the film's Wikipedia page.
    
    Returns:
        dict: A dictionary containing relevant film details.
    """
    response = requests.get(title_link, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, 'html.parser')
    infobox = soup.find("table", {"class": "infobox vevent"})

    if not infobox:
        return {}

    data = {}
    for row in infobox.find_all("tr"):
        header = row.find("th")
        value = row.find("td")

        if header and value:
            key = header.text.strip()
            val = value.text.strip()

            # Normalize field names and values
            if key in ["Directed by", "Produced by", "Box office", "Country", "Countries"]:
                if key in ["Directed by", "Produced by"]:
                    val = ', '.join(val.split('\n'))  # Separate names with commas
                if key in ["Country", "Countries"]:
                    key = "Country"  # Standardizing the key
                data[key] = val

    return data

def clean_text(text):
    """
    Cleans the extracted text by removing unwanted characters, extra spaces, and formatting inconsistencies.
    
    Parameters:
        text (str): The raw extracted text.
    
    Returns:
        str: The cleaned text.
    """
    if text is None:
        return ''
    text = re.sub(r'\[.*?\]', '', text)  # Remove reference brackets
    text = text.replace('\xa0billion', '').replace('billion', '').replace('\n', ' ')
    text = re.sub(r'[^0-9.]', '', text) if text.replace('.', '').isdigit() else text  # Keep only digits for box office values
    return text.strip()

# Locate the table containing the list of films
table = soup.find('table', {'class': 'wikitable'})
films_data = []

# Extract relevant information from each row
for row in table.find_all('tr')[1:]:  # Skipping header row
    columns = row.find_all(["th", "td"])
    if len(columns) < 5:
        continue

    title = columns[2].text.strip()
    year = columns[4].text.strip()

    title_link = columns[2].find("a")
    link = urljoin(BASE_URL, title_link["href"]) if title_link else None

    add_info = extract(link) if link else {}

    film = {
        "title": title,
        "year": year,
        **add_info
    }
    films_data.append(film)

# Clean the extracted data
cleaned_films = []
for film in films_data:
    cleaned_film = {key: clean_text(value) for key, value in film.items()}
    cleaned_films.append(cleaned_film)

# Print extracted films
for film in cleaned_films:
    print(film)


{'title': 'Avatar', 'year': '2009', 'Directed by': 'James Cameron', 'Produced by': 'James Cameron, Jon Landau', 'Country': 'United Kingdom United States', 'Box office': '$2.923'}
{'title': 'Avengers: Endgame', 'year': '2019', 'Directed by': 'Anthony RussoJoe Russo', 'Produced by': 'Kevin Feige', 'Country': 'United States', 'Box office': '$2.799'}
{'title': 'Avatar: The Way of Water', 'year': '2022', 'Directed by': 'James Cameron', 'Produced by': 'James Cameron, Jon Landau', 'Country': 'United States', 'Box office': '$2.320'}
{'title': 'Titanic', 'year': '1997', 'Directed by': 'James Cameron', 'Produced by': 'James Cameron, Jon Landau', 'Country': 'United States', 'Box office': '$2.264'}
{'title': 'Star Wars: The Force Awakens', 'year': '2015', 'Directed by': 'J. J. Abrams', 'Produced by': 'Kathleen Kennedy, J. J. Abrams, Bryan Burk', 'Country': 'United States', 'Box office': '$2.07'}
{'title': 'Avengers: Infinity War', 'year': '2018', 'Directed by': 'Anthony RussoJoe Russo', 'Produced 

In [2]:
import sqlite3

# Database file name
DB_FILE = "films.db"

def create_table():
    """
    Creates the 'films' table in the SQLite database if it does not already exist.
    """
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS films (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        release_year INTEGER,
        director TEXT,
        producers TEXT,
        box_office TEXT,
        country TEXT
    );
    """)

    conn.commit()
    conn.close()

def load_films_to_db(films):
    """
    Loads a list of films into the SQLite database.
    
    Parameters:
        films (list): A list of dictionaries containing film details.
    """
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    for film in films:
        cursor.execute("""
        INSERT INTO films (title, release_year, director, producers, box_office, country)
        VALUES (?, ?, ?, ?, ?, ?)
        """, (
            # Safely get values, even if the key is missing
            film.get("title", ""),
            film.get("year", None),
            film.get("Directed by", ""),
            film.get("Produced by", ""),
            film.get("Box office", ""),
            film.get("Country", "")
        ))

    # Commit all inserted records
    conn.commit()
    conn.close()

load_films_to_db(cleaned_films)
# Create the table before loading data
create_table()
